# TensorBoard

이 페이지에 있는 스크립트로 모델을 훈련하고 난 후 아래와 같이 실행해 주세요.

    tensorboard --logdir=logs/nn_logs --port=port_number
    
Tensorboard를 실행하실 때 꼭 logdir와 port번호를 확인 해 주세요. (port 번호는 port-forwarding 이용 시 사용)

In [1]:
import tensorflow as tf

# MINST 데이터 가져오기
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
def init_weights(shape, name):
    return tf.Variable(tf.random_normal(shape, stddev=0.01), name=name)

# This network is the same as the previous one except with an extra hidden layer + dropout
def model(X, w_h, w_h2, w_o, p_keep_input, p_keep_hidden):
    # Add layer name scopes for better graph visualization
    with tf.name_scope("layer1"):
        X = tf.nn.dropout(X, p_keep_input)
        h = tf.nn.relu(tf.matmul(X, w_h))
    with tf.name_scope("layer2"):
        h = tf.nn.dropout(h, p_keep_hidden)
        h2 = tf.nn.relu(tf.matmul(h, w_h2))
    with tf.name_scope("layer3"):
        h2 = tf.nn.dropout(h2, p_keep_hidden)
        return tf.matmul(h2, w_o)

trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

In [3]:
X = tf.placeholder("float", [None, 784], name="X")
Y = tf.placeholder("float", [None, 10], name="Y")

w_h = init_weights([784, 625], "w_h")
w_h2 = init_weights([625, 625], "w_h2")
w_o = init_weights([625, 10], "w_o")

# Add histogram summaries for weights
tf.histogram_summary("w_h_summ", w_h)
tf.histogram_summary("w_h2_summ", w_h2)
tf.histogram_summary("w_o_summ", w_o)

p_keep_input = tf.placeholder("float", name="p_keep_input")
p_keep_hidden = tf.placeholder("float", name="p_keep_hidden")
py_x = model(X, w_h, w_h2, w_o, p_keep_input, p_keep_hidden)

with tf.name_scope("cost"):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y))
    train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
    # Add scalar summary for cost
    tf.scalar_summary("cost", cost)

with tf.name_scope("accuracy"):
    correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(py_x, 1)) # Count correct predictions
    acc_op = tf.reduce_mean(tf.cast(correct_pred, "float")) # Cast boolean to float to average
    # Add scalar summary for accuracy
    tf.scalar_summary("accuracy", acc_op)

In [4]:
# 세션 설정
session_conf = tf.ConfigProto()
session_conf.gpu_options.allow_growth = True

# 그래프 실행
with tf.Session(config=session_conf) as sess:
    # create a log writer. run 'tensorboard --logdir=./logs/nn_logs'
    writer = tf.train.SummaryWriter("./logs/nn_logs", sess.graph) # for 0.8
    merged = tf.merge_all_summaries()

    # you need to initialize all variables
    tf.initialize_all_variables().run()

    for i in range(100):
        for start, end in zip(range(0, len(trX), 128), range(128, len(trX)+1, 128)):
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end],
                                          p_keep_input: 0.8, p_keep_hidden: 0.5})
        summary, acc = sess.run([merged, acc_op], feed_dict={X: teX, Y: teY,
                                          p_keep_input: 1.0, p_keep_hidden: 1.0})
        writer.add_summary(summary, i)  # Write summary
        print(i, acc)                   # Report the accuracy

0 0.9318
1 0.9596
2 0.9705
3 0.9726
4 0.9753
5 0.9785
6 0.9768
7 0.9778
8 0.9791
9 0.979
10 0.9798
11 0.9784
12 0.9794
13 0.9817
14 0.9814
15 0.9811
16 0.9809
17 0.9809
18 0.9816
19 0.9829
20 0.9815
21 0.9815
22 0.9833
23 0.9832
24 0.982
25 0.9819
26 0.9817
27 0.982
28 0.9831
29 0.9827
30 0.983
31 0.9833
32 0.984
33 0.9837
34 0.9834
35 0.9833
36 0.9843
37 0.9843
38 0.985
39 0.9847
40 0.9827
41 0.9838
42 0.9836
43 0.984
44 0.9845
45 0.9852
46 0.9832
47 0.9836
48 0.9839
49 0.9849
50 0.9837
51 0.9846
52 0.9825
53 0.9835
54 0.9842
55 0.9847
56 0.984
57 0.9846
58 0.9847
59 0.9857
60 0.9852
61 0.9845
62 0.9851
63 0.9841
64 0.9842
65 0.9852
66 0.9843
67 0.9849
68 0.9841
69 0.9834
70 0.984
71 0.9829
72 0.9849
73 0.985
74 0.985
75 0.9838
76 0.9841
77 0.9847
78 0.9841
79 0.9855
80 0.9851
81 0.9849
82 0.9847
83 0.9855
84 0.9842
85 0.9858
86 0.9855
87 0.9859
88 0.985
89 0.9849
90 0.984
91 0.9842
92 0.9844
93 0.985
94 0.9847
95 0.985
96 0.9834
97 0.9856
98 0.9854
99 0.9847
